In [377]:
import re

from tqdm import notebook as tqdm
import attr
import pandas as pd
import numpy as np
import seaborn as sns
import networkx as nx
import geopy

import umap
import hdbscan
from matplotlib import pyplot as plt
from community import community_louvain
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# Load data

In [2]:
df = pd.read_excel("data/FR-Questionnaire_francais.xlsx")

In [3]:
df.head(3)

,Horodateur,Comment restez-vous informé·e de la pandémie COVID-19 ?,Avez-vous l'impression que les gens autour de vous respectent les mesures politiques prises ?,Quels sont les sentiments que vous avez éprouvés depuis le début du confinement ?,Comment jugez-vous l'impact sur votre vie quotidienne ?,Que savez-vous sur le coronavirus ?,Quels endroits avez-vous visités la semaine dernière ?,Combien de fois êtes-vous sorti de chez vous la semaine dernière ?,Quel type de transport possédez-vous ?,"Au cours de la semaine dernière, avec combien de personnes avez-vous été en contact physique étroit (moins de 2 mètres, plus de 10 min), en dehors de votre foyer ?",...,Dans quel ville vivez-vous ?,Dans quel pays vivez-vous ?,Quelle est votre profession ?,Comment jugez-vous votre niveau d’exposition au coronavirus au travail ?,"Vous compris, combien de personnes vivent dans le même ménage que vous ?",Partagez-vous votre chambre ?,Unnamed: 34,Considérez-vous que vous êtes en bonne santé en ce moment ?,Quand avez-vous passé votre dernier examen de santé ?,Avez-vous passé un test pour le coronavirus au cours des derniers mois ?
0,3.27.2020 18:17:51,Médias sociaux,Plus ou moins,"Peur, Colère, Méfiance, Anxiété, Ennui",Significatif,Je suis bien informé·e,"Magasins d'alimentation ou supermarchés, Rue, ...",4 à 7 fois,Pas de véhicule,0,...,Londres,Angleterre,Étudiant·e,Moyen,0.0,Non,pas de rdv fréquent chez le médecin ou à l’hop...,Oui,NaN,Non
1,3.28.2020 13:57:29,"Journaux (physiques ou numériques), Médias soc...",Plutôt oui,"Peur, Tristesse, Dégoût, Méfiance, Anxiété, Ap...",Significatif,Je suis bien informé·e,Bureau (professionel),Entre 1 et 3 fois,Véhicule individuel,0,...,NaN,Suisse,Service public essentiel,Moyen,0.0,Non,Bien être au niveau physique mental et social,Non,2020-02-01 00:00:00,Non
2,3.27.2020 17:56:36,"Journaux (physiques ou numériques), Médias soc...",Plutôt oui,"Peur, Tristesse, Curiosité, Anxiété, Culpabili...",Significatif,Peu de choses,Magasins d'alimentation ou supermarchés,Entre 1 et 3 fois,Pas de véhicule,1,...,Paris,France,Autres employé·e·s,Faible,0.0,Non,Aucun symptôme du Covid 19 et pas de retour d'...,Oui,2019,Non


# Refine dataset

In [4]:
translation_strings = {}


def _(s):
    trans = translation_strings.get(s)
    if trans is None:
        return s
    else:
        return trans

In [5]:
def process_col(s):
    return s.lower().replace(" ", "_")


@attr.s
class Feature:
    name = attr.ib()


@attr.s
class CategoricalFeature(Feature):
    astype = attr.ib(default=float)
    
    def encode(self, df):
        drop_first = len(df[self.name].dropna().unique()) <= 2
        dummies = pd.get_dummies(
            df[self.name], drop_first=drop_first, prefix=self.name, prefix_sep="-"
        )
        return dummies.astype(self.astype).rename(columns=process_col)


@attr.s
class OrdinalFeature(Feature):
    levels = attr.ib(default=None)
    astype = attr.ib(default=float)

    def encode(self, df):
        renaming_dict = {
            name: ord for name, ord in zip(self.levels, range(len(self.levels)))
        }
        return df[self.name].astype(str).replace(renaming_dict).astype(self.astype)


@attr.s
class MultiCatFeature(Feature):
    sep = attr.ib(default=", ")
    astype = attr.ib(default=float)

    def encode(self, df):
        dummies = df[self.name].str.get_dummies(sep=self.sep).astype(self.astype)
        return dummies.rename(columns=process_col).rename(
            columns=lambda x: f"{self.name}-{x}"
        )


@attr.s
class NumFeature(Feature):
    max_val = attr.ib(default=np.inf)
    astype = attr.ib(default=float)
    
    def encode(self, df):
        def to_num(x):
            num_str = re.sub("[^.0-9]", "", str(x))
            if len(num_str) > 0:
                val = float(num_str)
                if val <= self.max_val:
                    return val

        return df[self.name].apply(to_num).astype(self.astype)


@attr.s
class CityFeature(Feature):
    def encode(self, df):
        pass


@attr.s
class CountryFeature(Feature):
    def encode(self, df):
        pass

In [6]:
lifestyle_df = df.drop(
    columns=[
        "Avez-vous l'impression que les gens autour de vous respectent les mesures politiques prises ?",
        "Quels sont les sentiments que vous avez éprouvés depuis le début du confinement ?",
        "Que savez-vous sur le coronavirus ?",
        "Horodateur",
        "Avez-vous accès aux ressources suivantes ?",
        "Utilisez-vous un masque et/ou des gants lorsque vous sortez ?",
#         "Décrivez ce que signifie pour vous un bon état de santé",
        "Quand avez-vous passé votre dernier examen de santé ?",
        "Avez-vous passé un test pour le coronavirus au cours des derniers mois ?",
        "Savez-vous que, pendant la durée du confinement, la meilleure façon de recevoir une aide médicale est d'appeler votre médecin, de travailler avec lui via les médias sociaux ou par appel vidéo ?",
        "Considérez-vous que vous êtes en bonne santé en ce moment ?",
    ]
)

In [7]:
features = {
    _("Comment restez-vous informé·e de la pandémie COVID-19 ?"): MultiCatFeature(
        name="information_channel", sep=", "
    ),
    _("Comment jugez-vous l'impact sur votre vie quotidienne ?"): OrdinalFeature(
        name="impact_on_life",
        levels=[_("Pas d'impact"), _("Léger"), _("Modéré"), _("Significatif")],
    ),
    _("Quels endroits avez-vous visités la semaine dernière ?"): MultiCatFeature(
        name="places_visited", sep=", "
    ),
    _(
        "Combien de fois êtes-vous sorti de chez vous la semaine dernière ?"
    ): OrdinalFeature(
        name="number_exits",
        levels=[
            _("Aucune"),
            _("Entre 1 et 3 fois"),
            _("4 à 7 fois"),
            _("Plus de 7 fois"),
        ],
    ),
    _("Quel type de transport possédez-vous ?"): CategoricalFeature(name="transport"),
    _(
        "Au cours de la semaine dernière, avec combien de personnes avez-vous "
        "été en contact physique étroit (moins de 2 mètres, plus de 10 min), en dehors de votre foyer ?"
    ): NumFeature(name="num_close_contact", max_val=500),
    _("Avez-vous voyagé au cours du dernier mois ?"): CategoricalFeature(
        name="recent_travel"
    ),
    _(
        "Êtes-vous bénévole dans une activité pour aider à lutter contre la pandémie ?"
    ): CategoricalFeature(name="volunteer"),
    _(
        "Partagez-vous l'une de ces installations avec des voisins ou des étrangers ?"
    ): MultiCatFeature(name="shared_installation", sep=", "),
    _("Comment travaillez-vous ou étudiez-vous maintenant ?"): CategoricalFeature(
        name="work_mode"
    ),
    _(
        "À quelle distance se trouve votre lieu d'approvisionnement alimentaire le plus proche ?"
    ): OrdinalFeature(
        name="distance_to_food_shop",
        levels=["Moins de 15 minutes", "15-30 minutes", "Plus de 30 minutes"],
    ),
    _("Comment vous procurez-vous de la nourriture ?"): MultiCatFeature(
        name="food_shop_type", sep=", "
    ),
    _("À quelle fréquence achetez-vous de la nourriture ?"): OrdinalFeature(
        name="shopping_frequency",
        levels=[
            _("Chaque jour"),
            _("Tous les 3 jours environ"),
            _("Chaque semaine"),
            _("Toutes les deux semaines"),
            _("Moins souvent"),
        ],
    ),
    _(
        "Depuis le début du confinement vous mangez en quantité : inférieure, identique, "
        "supérieure à d'habitude ?"
    ): OrdinalFeature(
        name="food_habit_change",
        levels=[_("Inférieure"), _("Identique"), _("Supérieure")],
    ),
    _(
        "Avec qui êtes-vous le plus en contact (virtuellement, en dehors de votre foyer) ?"
    ): CategoricalFeature(name="most_frequent_contact"),
    _(
        "Comment communiquez-vous à distance avec votre famille et vos amis ?"
    ): MultiCatFeature(name="relatives_contact_mean", sep=", "),
    _("Comment communiquez-vous à distance avec vos collègues ?"): MultiCatFeature(
        name="work_contact_mean", sep=", "
    ),
    _(
        "Depuis le début du confinement, à quelle fréquence communiquez-vous à distance avec "
        "vos amis et votre famille en dehors du foyer ?"
    ): OrdinalFeature(
        name="relatives_contact_frequency",
        levels=[
            _("Plusieurs fois par jour"),
            _("Tous les jours"),
            _("Tous les 3 jours environ"),
            _("Toutes les semaines"),
            _("Toutes les deux semaines"),
            _("Très rarement"),
            _("Pas du tout"),
        ],
    ),
    _(
        "Durant la semaine dernière, avec combien d'ami·e·s ou de proches avez-vous communiqué à distance ?"
    ): OrdinalFeature(
        name="num_relatives_contact",
        levels=[
            _("Aucun"),
            _("1-3"),
            _("4-6"),
            _("7-10"),
            _("11-20"),
            _("Plus de 20"),
        ],
    ),
    _("Quel est votre âge ?"): NumFeature(name="age"),
    _("Quel est votre genre ?"): CategoricalFeature(name="gender"),
    _("Quelle est votre profession ?"): CategoricalFeature(name="profession"),
    _(
        "Comment jugez-vous votre niveau d’exposition au coronavirus au travail ?"
    ): OrdinalFeature(name="work_exposure", levels=["Faible", "Moyen", "Élevé"]),
    _(
        "Vous compris, combien de personnes vivent dans le même ménage que vous ?"
    ): NumFeature(name="household_size"),
    _("Partagez-vous votre chambre ?"): CategoricalFeature(name="shared_room"),
}

In [8]:
feature_names = [f.name for f in features.values()]
lifestyle_df = lifestyle_df.rename(
    columns={old: new for old, new in zip(features.keys(), feature_names)}
)

In [9]:
dfs = []
from IPython.display import display

for feature in features.values():
    feature_df = feature.encode(lifestyle_df)
    dfs.append(feature_df)

processed_data = pd.concat(dfs, axis=1)
processed_data

,information_channel-amis_ou_famille,information_channel-journaux_(physiques_ou_numériques),information_channel-médias_sociaux,information_channel-médias_sociaux_,information_channel-médias_sociaux/telegram,information_channel-télévision/radio,impact_on_life,places_visited-bureau_(professionel),places_visited-hôpital,places_visited-magasins_d'alimentation_ou_supermarchés,...,profession-service_public_essentiel,profession-travailleur·euse_de_la_santé,profession-vendeur·euse_(toujours_en_activité),profession-étudiant·e,work_exposure,household_size,shared_room-non,shared_room-oui_avec_un·e_partenaire,shared_room-oui_avec_un·e_proche,"shared_room-oui,_avec_un·e_colocataire"
0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
1,0.0,1.0,1.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,0.0,1.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766,0.0,1.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0
767,1.0,1.0,1.0,0.0,0.0,1.0,3.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,3.0,1.0,0.0,0.0,0.0
768,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0
769,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0


Sanity check:

In [10]:
for col in processed_data.columns:
    print(col, processed_data[col].unique())

information_channel-amis_ou_famille [0. 1.]
information_channel-journaux_(physiques_ou_numériques) [0. 1.]
information_channel-médias_sociaux [1. 0.]
information_channel-médias_sociaux_ [0. 1.]
information_channel-médias_sociaux/telegram [0. 1.]
information_channel-télévision/radio [0. 1.]
impact_on_life [ 3.  2.  1.  0. nan]
places_visited-bureau_(professionel) [0. 1.]
places_visited-hôpital [0. 1.]
places_visited-magasins_d'alimentation_ou_supermarchés [1. 0.]
places_visited-maison_d'un_parent [0. 1.]
places_visited-rue [1. 0.]
places_visited-transports_publics [0. 1.]
places_visited-espace_public [0. 1.]
places_visited-parc [1. 0.]
places_visited-pharmacie [0. 1.]
number_exits [ 2.  1.  0.  3. nan]
transport-pas_de_véhicule [1. 0.]
transport-véhicule_individuel [0. 1.]
transport-véhicule_partagé [0. 1.]
num_close_contact [  0.   1.  15.   5.   3.   2.  nan  20.   4.   6.  10.  50.   8.   7.
  28.  30. 100. 500.  40.  18.]
recent_travel-oui [0. 1.]
volunteer-oui [0. 1.]
shared_instal

In [78]:
compute_locations = True

if compute_locations:
    locations, latitude, longitude = [], [], []
    for i, row in tqdm.tqdm(list(lifestyle_df[["Dans quel ville vivez-vous ?", "Dans quel pays vivez-vous ?"]].iterrows())):
        query = ''
        if type(row["Dans quel ville vivez-vous ?"]) is str:
            query += str(row["Dans quel ville vivez-vous ?"]) + ', '
        if type(row["Dans quel pays vivez-vous ?"]) is str:
            query += str(row["Dans quel pays vivez-vous ?"]) + ', '
        
        geocoder = geopy.geocoders.Nominatim(user_agent="pdm", timeout=10)
        query_result = geocoder.geocode(query)
        address = query_result.address if query_result is not None else None
        locations.append(address)
        lat = query_result.latitude if query_result is not None else None
        latitude.append(lat)
        lon = query_result.latitude if query_result is not None else None
        longitude.append(lon)
        
    df_location = pd.DataFrame({'location': location, 'lat': latitude, 'lon': longitude})
    df_location.to_csv('data/df_location.csv')

GeocoderQuotaExceeded: HTTP Error 429: Too Many Requests

## Build social network and cluster communities

In [61]:
matrix = np.zeros((len(processed_data), len(processed_data)))

for i, row_i in tqdm.tqdm(list(processed_data.iterrows())):
    for j, row_j in processed_data.iterrows():
        if j > i:
            score = np.sum((row_i.values == row_j.values))
            matrix[i][j] = score
            matrix[j][i] = score

In [66]:
np.min(matrix[matrix > 0]), np.max(matrix), np.mean(matrix[matrix > 0]), np.median(matrix[matrix > 0])

(35.0, 79.0, 55.61974497616521, 56.0)

In [414]:
# Normalisation
processed_matrix = (matrix >= np.percentile(matrix[matrix > 0], 95)).astype('bool')

In [415]:
G = nx.from_numpy_matrix(processed_matrix)

In [416]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 771
Number of edges: 17682
Average degree:  45.8677


In [429]:
nx.write_gexf(G, "social_graph.gexf")

In [417]:
partition = community_louvain.best_partition(G, random_state=0)

In [418]:
df_communities = processed_data.copy()
df_communities['community'] = pd.Series(partition)

In [419]:
list_communities = df_communities['community'].value_counts()
list_communities

0     224
3     221
6     205
4      69
2      29
19      5
10      1
1       1
5       1
7       1
8       1
9       1
23      1
22      1
12      1
13      1
14      1
15      1
16      1
17      1
18      1
20      1
21      1
11      1
Name: community, dtype: int64

In [456]:
new_df = df.copy()
community = pd.Series(partition)
new_df['community'] = pd.Series(partition)

In [457]:
for isolated in list_communities[list_communities <= 5].index:
    new_df['community'][new_df['community'] == isolated] = -1

<ipython-input-457-5e895ca46a86>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['community'][new_df['community'] == isolated] = -1


In [458]:
new_df['community'].value_counts()

 0    224
 3    221
 6    205
 4     69
 2     29
-1     23
Name: community, dtype: int64

In [459]:
new_df.to_excel('data/FR_Questionnaire_communities.xlsx')

## Analyse communities

### Outliers

In [428]:
for isolated in list_communities[list_communities == 1].index:
    individual = df[df_communities['community'] == isolated]
    print('Individual', individual.index[0]+2)

Individual 118
Individual 3
Individual 14
Individual 25
Individual 34
Individual 96
Individual 769
Individual 753
Individual 368
Individual 463
Individual 488
Individual 582
Individual 617
Individual 638
Individual 647
Individual 732
Individual 733
Individual 119


### Main communities

In [420]:
df_main_communities = df_communities[(df_communities['community'] == 0) | (
    df_communities['community'] == 6) | (
    df_communities['community'] == 3) | (
    df_communities['community'] == 4) | (
    df_communities['community'] == 2)].dropna()

In [421]:
X, y = shuffle(df_main_communities.drop(columns='community'), df_main_communities['community'], 
               random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [422]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
lda.score(X_test, y_test)

0.905511811023622

In [423]:
community_size = []
for i in range(3):
    community_size.append(np.sum((df_main_communities['community'] == i)))
print('Chance lies at', np.max(community_size)/len(df_main_communities))

Chance lies at 0.27848101265822783


In [424]:
X, y = df_main_communities.drop(columns='community'), df_main_communities['community']

lda = LinearDiscriminantAnalysis()
lda.fit_transform(X, y)
persona = lda.decision_function(df_main_communities.drop(columns='community'))

In [425]:
lda.fit_transform(X, y)

array([[ 0.27547191,  3.54239474,  6.26518644, -0.28427224],
       [-0.76558582,  3.30106564,  4.49970065,  0.84868169],
       [-0.07227963,  2.77406474,  4.95927386,  1.5039095 ],
       ...,
       [ 0.83711483, -2.28582539,  1.19658986, -0.97696138],
       [ 1.18872963, -2.8269773 ,  0.63873668, -1.4797107 ],
       [ 4.17507545,  1.17557322, -1.17952514, -0.46954952]])

In [426]:
probabilities = lda.predict_proba(X)

In [449]:
for i in range(5):
    print('Typical individual for community', y.iloc[np.argmax(np.abs(probabilities[:,i]))], ':',
          X.iloc[np.argmax(np.abs(probabilities[:,i]))].name+2)
    
    for top_question in np.argsort(np.abs(lda.coef_[i,:]))[-10:]:
        print('   Question:', X.columns[top_question])
        print('   Impact:', lda.coef_[i,:][top_question])

Typical individual for community 0 : 497
   Question: shared_room-oui_avec_un·e_partenaire
   Impact: -1.745983036486568
   Question: shared_installation-dortoir
   Impact: 1.755907309805359
   Question: places_visited-hôpital
   Impact: -1.837643609842742
   Question: places_visited-pharmacie
   Impact: 1.9097088514616007
   Question: shared_room-oui,_avec_un·e_colocataire
   Impact: 2.329122373392456
   Question: transport-véhicule_individuel
   Impact: -2.4121996193392463
   Question: gender-autre
   Impact: -2.488384821477021
   Question: profession-livreur·euse
   Impact: -2.61187816061832
   Question: shared_room-non
   Impact: 3.1537563171853544
   Question: profession-étudiant·e
   Impact: 4.769906432464031
Typical individual for community 2 : 195
   Question: shared_installation-toilettes
   Impact: 7.100160293200204
   Question: profession-travailleur·euse_de_la_santé
   Impact: 7.334309686955643
   Question: shared_room-oui,_avec_un·e_colocataire
   Impact: -8.74014512801644

## Some initial cluster analysis

In [ ]:
clean_data = processed_data.dropna()
data = clean_data.drop(columns=[
    "age",
    "gender-autre",
    "gender-f",
    "gender-m",
])

reducer = umap.UMAP(random_state=1, n_neighbors=3, n_components=10)
embedding = reducer.fit_transform(data)

fig, ax = plt.subplots(figsize=(12, 10))
sns.scatterplot(
    x=embedding[:, 0], y=embedding[:, 1], hue=clean_data.age,
    ax=ax
)
plt.setp(ax, xticks=[], yticks=[])
plt.show()

In [ ]:
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=25,
).fit(embedding)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
sns.scatterplot(
    x=embedding[:, 0], y=embedding[:, 1],
    hue=clusterer.probabilities_,
    style=clusterer.labels_,
    ax=ax
)
plt.setp(ax, xticks=[], yticks=[])
plt.show()